#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [4]:
#names here
# Tarun Gunampalli - tg25522
# Joey Huang - jth3785

In [5]:
# !pip install pandas
# !pip install numpy

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [6]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2

In [7]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [8]:
def get_data(filename):
    data = []
#     your code goes here
    file_data = pd.read_csv(filename, header=None)
    for (index, features) in file_data.iterrows():
        data.append(DataPoint(features[19], features[:19]))
    return data

In [9]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [10]:
def make_prediction(tree_root, data_point):
#     your code goes here
    if tree_root.is_leaf:
        return tree_root.prediction
    left = data_point.features[tree_root.feature_idx] < tree_root.thresh_val
    child = tree_root.left_child if left else tree_root.right_child
    return make_prediction(child, data_point)

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [11]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
#     your code goes here
    for data_point in data:
        if data_point.features[feature_idx] < threshold:
            left_split.append(data_point)
        else:
            right_split.append(data_point)
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [12]:
def calc_entropy(data):
    entropy = 0.0
#     your code goes here
    if len(data) == 0:
        return entropy
    diabetic = sum(d.label for d in data) / len(data)
    non_diabetic = 1 - diabetic
    if diabetic != 0:
        entropy -= diabetic * log2(diabetic)
    if non_diabetic != 0:
        entropy -= non_diabetic * log2(non_diabetic)
    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [13]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
#     your code goes here
    imp_p = calc_entropy(data)
    feature_sorted = sorted(data, key=lambda d: d.features[feature_idx])
    left_split = feature_sorted[:1]
    right_split = feature_sorted[1:]
    while len(right_split) >= 1:
        lower = left_split[-1].features[feature_idx]
        upper = right_split[0].features[feature_idx]
        if lower != upper:
            thresh = lower + (upper - lower) / 2
            imp_l = calc_entropy(left_split)
            imp_r = calc_entropy(right_split)
            gain = imp_p - (len(left_split) / len(data)) * imp_l - (len(right_split) / len(data)) * imp_r
            if gain > best_info_gain or best_thresh is None:
                best_info_gain = gain
                best_thresh = thresh
        left_split.append(right_split.pop(0))
    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [14]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
#     your code goes here
    best_gain = 0.0
    for feature_idx in range(19):
        (gain, thresh) = calc_best_threshold(data, feature_idx)
        if gain > best_gain or best_feature is None or best_thresh is None:
            best_gain = gain
            best_feature = feature_idx
            best_thresh = thresh
    if best_thresh is None:
        print(best_gain)
    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [15]:
def create_leaf_node(data):
#     your code goes here
    node = TreeNode()
    node.is_leaf = True
    diabetic = sum(d.label for d in data)
    non_diabetic = len(data) - diabetic
    node.prediction = 1 if diabetic >= non_diabetic else 0
    return node

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [16]:
def create_decision_tree(data, max_levels):
#     your code goes here
    diabetic = sum(d.label for d in data)
    if max_levels == 1 or len(data) < 2 or diabetic == len(data) or diabetic == 0:
        return create_leaf_node(data)
    node = TreeNode()
    node.is_leaf = False
    (feature_idx, threshold) = identify_best_split(data)
    node.feature_idx = feature_idx
    node.thresh_val = threshold
    (left_split, right_split) = split_dataset(data, feature_idx, threshold)
    if len(left_split) == 0 or len(right_split) == 0:
        node = create_leaf_node(data)
    else:
        node.left_child = create_decision_tree(left_split, max_levels - 1)
        node.right_child =  create_decision_tree(right_split, max_levels - 1)
    return node

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [17]:
def calc_accuracy(tree_root, data):
#     your code goes here
    correct = sum(1 for data_point in data if make_prediction(tree_root, data_point) == data_point.label) / len(data)
    return correct

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [2]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")

for i in range(5):
    # partition data into train_set and test_set
    test_start = int(len(d) * i / 5)
    test_end = int(len(d) * (i + 1) / 5)
    train_set = d[:test_start] + d[test_end:]
    test_set = d[test_start:test_end]

    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # the timer is just for fun! you will NOT be graded on runtime
    start = time.time()

    # create the decision tree
    tree = create_decision_tree(train_set, 10)

    end = time.time()
    print ('Time taken:', end - start)

    # calculate the accuracy of the tree
    accuracy = calc_accuracy(tree, test_set)
    print ('The accuracy on the test set is ', str(accuracy * 100.0))
    #t.printTree()

NameError: name 'get_data' is not defined